# Mermaid Diagram Examples

Test `graph.to_mermaid()` rendering — fully offline, no data sent externally.

- **JupyterLab 4.1+**: uses native `text/vnd.mermaid` MIME type
- **VSCode / older Jupyter**: uses bundled `beautiful-mermaid` JS (rendered locally in iframe)
- **`print()`**: raw Mermaid text you can paste anywhere

In [1]:
from hypergraph import Graph, node, route, ifelse, END

## 1. Simple Pipeline

In [2]:
@node(output_name="cleaned")
def clean(raw_text: str) -> str:
    return raw_text.strip().lower()


@node(output_name="tokens")
def tokenize(cleaned: str) -> list[str]:
    return cleaned.split()


@node(output_name="embedding")
def embed(tokens: list[str]) -> list[float]:
    return [0.1] * len(tokens)


pipeline = Graph(nodes=[clean, tokenize, embed])
pipeline.to_mermaid()

flowchart TD
    %% Inputs
    input_raw_text(["raw_text"])
    %% Nodes
    clean["clean"]
    tokenize["tokenize"]
    embed["embed"]
    %% Edges
    input_raw_text --> clean
    clean -->|cleaned| tokenize
    tokenize -->|tokens| embed

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef input fill:#E0F7FA,stroke:#0097A7,stroke-width:2px,color:#004D40
    class clean,tokenize,embed function
    class input_raw_text input

## 2. With Type Annotations

In [3]:
pipeline.to_mermaid(show_types=True)

flowchart TD
    %% Inputs
    input_raw_text(["raw_text: str"])
    %% Nodes
    clean["clean<br/>cleaned: str"]
    tokenize["tokenize<br/>tokens: list[str]"]
    embed["embed<br/>embedding: list[float]"]
    %% Edges
    input_raw_text --> clean
    clean -->|cleaned| tokenize
    tokenize -->|tokens| embed

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef input fill:#E0F7FA,stroke:#0097A7,stroke-width:2px,color:#004D40
    class clean,tokenize,embed function
    class input_raw_text input

## 3. Branching (ifelse)

In [4]:
@node(output_name="cached_result")
def use_cache(query: str) -> str:
    return "cached answer"


@node(output_name="fresh_result")
def compute_fresh(query: str) -> str:
    return "fresh answer"


@ifelse(when_true="use_cache", when_false="compute_fresh")
def check_cache(query: str) -> bool:
    return query in ["hello", "world"]


Graph(nodes=[check_cache, use_cache, compute_fresh]).to_mermaid()

flowchart TD
    %% Inputs
    input_query(["query"])
    %% Nodes
    check_cache{"check_cache"}
    use_cache["use_cache"]
    compute_fresh["compute_fresh"]
    %% Edges
    input_query --> check_cache
    input_query --> use_cache
    input_query --> compute_fresh
    check_cache -->|True| use_cache
    check_cache -->|False| compute_fresh

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef branch fill:#E0F7FA,stroke:#00ACC1,stroke-width:2px,color:#006064
    classDef input fill:#E0F7FA,stroke:#0097A7,stroke-width:2px,color:#004D40
    class check_cache branch
    class use_cache,compute_fresh function
    class input_query input

## 4. Agentic Loop (cycle with END)

In [5]:
@node(output_name="response")
def generate(messages: list[dict]) -> str:
    return "I can help with that!"


@node(output_name="messages")
def accumulate(messages: list[dict], response: str) -> list[dict]:
    return messages + [{"role": "assistant", "content": response}]


@route(targets=["generate", END])
def should_continue(messages: list[dict]) -> str:
    if len(messages) > 5:
        return END
    return "generate"


Graph(nodes=[generate, accumulate, should_continue]).to_mermaid()

flowchart TD
    %% Nodes
    generate["generate"]
    accumulate["accumulate"]
    should_continue{"should_continue"}
    __end__(["End"])
    %% Edges
    generate -->|response| accumulate
    accumulate -->|messages| generate
    accumulate -->|messages| should_continue
    should_continue --> generate
    should_continue --> __end__

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef branch fill:#E0F7FA,stroke:#00ACC1,stroke-width:2px,color:#006064
    classDef end fill:#E8F5E9,stroke:#43A047,stroke-width:2px,color:#1B5E20
    class should_continue branch
    class __end__ end
    class generate,accumulate function

## 5. Nested Graph (collapsed vs expanded)

In [6]:
@node(output_name="cleaned")
def clean_text(text: str) -> str:
    return text.strip()


@node(output_name="normalized")
def normalize(cleaned: str) -> str:
    return cleaned.lower()


preprocess = Graph(nodes=[clean_text, normalize], name="preprocess")


@node(output_name="result")
def analyze(normalized: str) -> dict:
    return {"length": len(normalized)}


workflow = Graph(nodes=[preprocess.as_node(), analyze])
workflow.to_mermaid(depth=0)

flowchart TD
    %% Inputs
    input_text(["text"])
    %% Nodes
    preprocess[["preprocess"]]
    analyze["analyze"]
    %% Edges
    input_text --> preprocess
    preprocess -->|normalized| analyze

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef graph fill:#FFF8E1,stroke:#FFB300,stroke-width:2px,color:#E65100
    classDef input fill:#E0F7FA,stroke:#0097A7,stroke-width:2px,color:#004D40
    class analyze function
    class preprocess graph
    class input_text input

In [7]:
workflow.to_mermaid(depth=1)

flowchart TD
    %% Inputs
    input_text(["text"])
    %% Nodes
    subgraph preprocess ["preprocess"]
        preprocess__clean_text["clean_text"]
        preprocess__normalize["normalize"]
    end
    analyze["analyze"]
    %% Edges
    input_text --> preprocess__clean_text
    preprocess__normalize -->|normalized| analyze
    preprocess__clean_text -->|cleaned| preprocess__normalize

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef input fill:#E0F7FA,stroke:#0097A7,stroke-width:2px,color:#004D40
    class preprocess__clean_text,preprocess__normalize,analyze function
    class input_text input

## 6. Direction and Separate Outputs

In [8]:
pipeline.to_mermaid(direction="LR")

flowchart LR
    %% Inputs
    input_raw_text(["raw_text"])
    %% Nodes
    clean["clean"]
    tokenize["tokenize"]
    embed["embed"]
    %% Edges
    input_raw_text --> clean
    clean -->|cleaned| tokenize
    tokenize -->|tokens| embed

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef input fill:#E0F7FA,stroke:#0097A7,stroke-width:2px,color:#004D40
    class clean,tokenize,embed function
    class input_raw_text input

In [9]:
pipeline.to_mermaid(separate_outputs=True)

flowchart TD
    %% Inputs
    input_raw_text(["raw_text"])
    %% Nodes
    clean["clean"]
    tokenize["tokenize"]
    embed["embed"]
    data_clean_cleaned[/"cleaned"/]
    data_tokenize_tokens[/"tokens"/]
    data_embed_embedding[/"embedding"/]
    %% Edges
    input_raw_text --> clean
    clean --> data_clean_cleaned
    tokenize --> data_tokenize_tokens
    embed --> data_embed_embedding
    data_clean_cleaned -->|cleaned| tokenize
    data_tokenize_tokens -->|tokens| embed

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef input fill:#E0F7FA,stroke:#0097A7,stroke-width:2px,color:#004D40
    classDef data fill:#ECEFF1,stroke:#78909C,stroke-width:2px,color:#37474F
    class data_clean_cleaned,data_tokenize_tokens,data_embed_embedding data
    class clean,tokenize,embed function
    class input_raw_text input

## 7. Raw Source

`print()` gives you the raw Mermaid text — paste into any Mermaid-compatible tool.

In [10]:
print(pipeline.to_mermaid())

flowchart TD
    %% Inputs
    input_raw_text(["raw_text"])
    %% Nodes
    clean["clean"]
    tokenize["tokenize"]
    embed["embed"]
    %% Edges
    input_raw_text --> clean
    clean -->|cleaned| tokenize
    tokenize -->|tokens| embed

    %% Styling
    classDef function fill:#E8EAF6,stroke:#5C6BC0,stroke-width:2px,color:#283593
    classDef input fill:#E0F7FA,stroke:#0097A7,stroke-width:2px,color:#004D40
    class clean,tokenize,embed function
    class input_raw_text input


## 8. CLI / Terminal Rendering (ASCII)

`to_ascii()` renders diagrams as Unicode box-drawing art — great for CLI tools and terminal output.

Requires: `npm install -g beautiful-mermaid`

In [ ]:
print(pipeline.to_mermaid().to_ascii())

In [ ]:
print(pipeline.to_mermaid(direction="LR").to_ascii())